## Dependencies

In [39]:
!pip install gensim

In [40]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Masking, Embedding, Bidirectional, LSTM, Attention, Dropout, Dense
from tensorflow.keras import Model, Input
import gensim as gs
import ast
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [41]:
#! check if tensorflow is using GPU
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print(f"TensorFlow is using {len(gpus)} GPU(s).")
    for gpu in gpus:
        print(f"GPU: {gpu.name}")
else:
    print("TensorFlow is not using any GPUs.")

TensorFlow is using 1 GPU(s).
GPU: /physical_device:GPU:0


In [42]:
def load_data_labels(data_path, labels_path):
    with open(data_path, 'r') as f:
        data = [ast.literal_eval(line.strip()) for line in f]
    with open(labels_path, 'r') as f:
        labels = [ast.literal_eval(line.strip()) for line in f]
    return data, labels

In [43]:
data, labels = load_data_labels('training_data_processed.txt', 'train_order_category_labels.txt')
data2, labels2 = load_data_labels('synthetic_orders.txt', 'synthetic_labels.txt')
data.extend(data2)
labels.extend(labels2)
print(len(data))
print(len(labels))
print(data[:5])
print(labels[:5])

60000
60000
[['two', '20', 'fl', 'ounce', 'diet', 'sprite', 'in', 'can', 'and', 'a', 'dr', 'peper', 'and', 'three', 'twoliter', 'pepsi'], ['can', 'i', 'have', 'one', 'personal', 'size', 'pizza', 'with', 'no', 'american', 'cheese'], ['a', 'bottle', 'of', 'ice', 'tea', 'and', 'three', 'fantas', 'and', 'four', '500milliliter', 'sprite'], ['three', 'pizza', 'no', 'american', 'cheese', 'and', 'five', 'medium', 'pizza', 'with', 'green', 'olive'], ['four', 'party', 'size', 'pizza', 'with', 'only', 'a', 'little', 'barbecue', 'sauce']]
[[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1], [2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1], [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [44]:
dev_data, dev_labels = load_data_labels('dev_data_processed.txt', 'dev_order_category_labels.txt')


In [45]:
pretrained_model = gs.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [46]:
#! get v Aand replace unknown words with unk token
def process_sentence(sentence, model):
    for i, word in enumerate(sentence):
        if word not in model and word  != 'a':
            sentence[i] = 'unk'
    return sentence

data = [process_sentence(sentence, pretrained_model) for sentence in data]
print(data[:5])
vocab=set()
for sentence in data:
    vocab.update(sentence)
#! get word index for each word in vocab
word2idx = {word: idx for idx, word in enumerate(vocab)}

[['two', 'unk', 'fl', 'ounce', 'diet', 'sprite', 'in', 'can', 'unk', 'a', 'dr', 'unk', 'unk', 'three', 'unk', 'pepsi'], ['can', 'i', 'have', 'one', 'personal', 'size', 'pizza', 'with', 'no', 'american', 'cheese'], ['a', 'bottle', 'unk', 'ice', 'tea', 'unk', 'three', 'unk', 'unk', 'four', 'unk', 'sprite'], ['three', 'pizza', 'no', 'american', 'cheese', 'unk', 'five', 'medium', 'pizza', 'with', 'green', 'olive'], ['four', 'party', 'size', 'pizza', 'with', 'only', 'a', 'little', 'barbecue', 'sauce']]


In [47]:
embedding_dim=300
input_dim=len(vocab)
output_dim=3
max_length=100

In [48]:
for tokens in dev_data:
    for i,word in enumerate(tokens):
        if word not in vocab!= 'a':
            tokens[i] = 'unk'
X_d=[[word2idx[word] for word in sentence] for sentence in dev_data]
X_d=pad_sequences(X_d, maxlen=max_length, padding='post', value=-1)
Y_d=pad_sequences(dev_labels, maxlen=max_length, padding='post', value=2)

In [49]:
#! get embeddings matrix
def get_embeddings_matrix(model, vocab):
    vocab_size = len(vocab)
    embedding_dim = model.vector_size
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for i, word in enumerate(vocab):
        if word in model:
            embedding_matrix[i] = model[word]
        elif word == 'a':
            embedding_matrix[i] = model['one']
    return embedding_matrix
embedding_matrix = get_embeddings_matrix(pretrained_model, vocab)

In [50]:
#! replace words with their index in vocab and pad sentences
X=[[word2idx[word] for word in sentence] for sentence in data]
X=pad_sequences(X, maxlen=max_length, padding='post', value=-1)
Y=pad_sequences(labels, maxlen=max_length, padding='post', value=2)

In [51]:
# input_seq = Input(shape=(None,), dtype=tf.int32)
# masked_input = Masking(mask_value=-1)(input_seq)

# embedding = Embedding(input_dim=input_dim, output_dim=embedding_dim, weights=[embedding_matrix], trainable=True, mask_zero=False)(masked_input)

# lstm_output = Bidirectional(LSTM(32, return_sequences=True))(embedding)

# query = LSTM(32, return_sequences=True)(lstm_output)  
# key = LSTM(32, return_sequences=True)(lstm_output)    
# value = LSTM(32, return_sequences=True)(lstm_output)  

# attention_output = Attention(use_scale=True, causal=False)([query, key, value])

# dropout_output = Dropout(0.8)(attention_output)
# output = Dense(output_dim, activation='softmax')(dropout_output)

# model = Model(inputs=input_seq, outputs=output)

# optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

# model.compile(
#     optimizer=optimizer,
#     loss='sparse_categorical_crossentropy',
#     metrics=['accuracy']
# )

# model.summary()

# lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
#     monitor='val_loss',
#     factor=0.5,
#     patience=5,
#     min_lr=1e-9,
#     verbose=1
# )

In [52]:
# model.fit(
#     X, 
#     Y, 
#     validation_data=(X_d, Y_d),  # Add validation data
#     callbacks=[lr_scheduler], 
#     epochs=25,
#     batch_size=1024
# )


In [79]:
#! model
model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=-1),  #! Masking layer to handle the padded values
    tf.keras.layers.Embedding(input_dim=input_dim, output_dim=embedding_dim, weights=[embedding_matrix], trainable=True,mask_zero=False),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(output_dim, activation='softmax')
])
## dropout->.8 , layers->48 best acc->70%
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',           #! Monitor the validation loss (you can use 'accuracy' or another metric)
    factor=0.5,               #! Factor by which the learning rate will be reduced
    patience=5,               #! Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-9,              #! Lower bound on the learning rate
    verbose=1                 #! Print a message when the learning rate is reduced
)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.003)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [80]:
model.fit(
    X, 
    Y, 
    validation_data=(X_d, Y_d),  # Add validation data
    callbacks=[lr_scheduler], 
    epochs=5,
    batch_size=128
)

Epoch 1/5
469/469 [==============================] - 12s 20ms/step - loss: 0.0370 - accuracy: 0.9922 - val_loss: 0.0693 - val_accuracy: 0.9872 - lr: 0.0030
Epoch 2/5
469/469 [==============================] - 9s 19ms/step - loss: 0.0046 - accuracy: 0.9987 - val_loss: 0.0846 - val_accuracy: 0.9874 - lr: 0.0030
Epoch 3/5
469/469 [==============================] - 9s 19ms/step - loss: 0.0026 - accuracy: 0.9992 - val_loss: 0.1058 - val_accuracy: 0.9866 - lr: 0.0030
Epoch 4/5
469/469 [==============================] - 9s 19ms/step - loss: 0.0018 - accuracy: 0.9994 - val_loss: 0.1119 - val_accuracy: 0.9871 - lr: 0.0030
Epoch 5/5
469/469 [==============================] - 9s 19ms/step - loss: 0.0014 - accuracy: 0.9996 - val_loss: 0.1356 - val_accuracy: 0.9858 - lr: 0.0030


In [81]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_5 (Masking)         (None, 100)               0         
                                                                 
 embedding_5 (Embedding)     (None, 100, 300)          101100    
                                                                 
 bidirectional_5 (Bidirectio  (None, 100, 64)          85248     
 nal)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 100, 64)           0         
                                                                 
 dense_5 (Dense)             (None, 100, 3)            195       
                                                                 
Total params: 186,543
Trainable params: 186,543
Non-trainable params: 0
________________________________________________

## Evaluate model on training data

In [82]:
preds_train = model.predict(X)
preds_train = np.argmax(preds_train, axis=-1)

1875/1875 [==============================] - 16s 8ms/step


In [83]:
count = 0
for i in range(len(X)):
    mask=X[i]!=-1
    if np.all(preds_train[i][mask]==Y[i][mask]):
        count+=1
print(f"Accuracy on training data: {count/len(X)}")

Accuracy on training data: 0.9976833333333334


In [84]:
# # Predict on the development set
# preds_dev = Bidirectional_LSTM_model.predict(X)
# preds_dev = np.argmax(preds_dev, axis=-1)

# count = 0  # Count of completely correct sequences
# last_index_error_count = 0  # Count of sequences with only last index error

# for i in range(len(data)):
#     # Get original sequence lengths before padding
#     original_length = len(data[i])
    
#     # Extract predictions and true labels for the original sequence
#     pred_seq = preds_dev[i][:original_length]
#     true_seq = labels[i]  # Original labels are unpadded
    
#     # Check if the sequence is entirely correct
#     if (pred_seq == true_seq).all():
#         count += 1
#     else:
#         # Check if only the last index is incorrect
#         if (pred_seq[:-1] == true_seq[:-1]).all() and (pred_seq[-1] != true_seq[-1]):
#             last_index_error_count += 1
        
#         # Print debug information for mismatches
#         print(f"Index with mismatch: {i}")
#         print(f"Predicted: {pred_seq}")
#         print(f"True:      {true_seq}")
#         print("--------------------------------------------")

# # Print results
# print(f"Sequences with only last index error: {last_index_error_count}")


## Evaluate model on dev data

In [85]:
# dev_data, dev_labels = load_data_labels('dev_data_processed.txt', 'dev_order_category_labels.txt')
# for tokens in dev_data:
#     for i,word in enumerate(tokens):
#         if word not in vocab!= 'a':
#             tokens[i] = 'unk'
# X_d=[[word2idx[word] for word in sentence] for sentence in dev_data]
# X_d=pad_sequences(X_d, maxlen=max_length, padding='post', value=-1)
# Y_d=pad_sequences(dev_labels, maxlen=max_length, padding='post', value=2)

In [86]:
# Predict on the development set
preds_dev = model.predict(X_d)
preds_dev = np.argmax(preds_dev, axis=-1)

count = 0  # Count of completely correct sequences
last_index_error_count = 0  # Count of sequences with only last index error

for i in range(len(dev_data)):
    # Get original sequence lengths before padding
    original_length = len(dev_data[i])
    
    # Extract predictions and true dev_labels for the original sequence
    pred_seq = preds_dev[i][:original_length]
    true_seq = dev_labels[i]  # Original dev_labels are unpadded
    
    # Check if the sequence is entirely correct
    if (pred_seq == true_seq).all():
        count += 1
    else:
        # Check if only the last index is incorrect
        if (pred_seq[:-1] == true_seq[:-1]).all() and (pred_seq[-1] != true_seq[-1]):
            last_index_error_count += 1
        
        # Print debug information for mismatches
        print(f"Index with mismatch: {i}")
        print(f"Predicted: {pred_seq}")
        print(f"True:      {true_seq}")
        print("--------------------------------------------")

# Print results
print(f"Accuracy on dev_data: {count / len(dev_data):.4f}")
print(f"Sequences with only last index error: {last_index_error_count}")


11/11 [==============================] - 1s 10ms/step
Index with mismatch: 3
Predicted: [2 2 0 2 0 0 0 0 0 0]
True:      [2, 2, 2, 2, 0, 0, 0, 0, 0, 0]
--------------------------------------------
Index with mismatch: 5
Predicted: [2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 1 1]
True:      [2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1]
--------------------------------------------
Index with mismatch: 25
Predicted: [2 2 2 2 2 2 0 0 0 0 0 2]
True:      [2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2]
--------------------------------------------
Index with mismatch: 29
Predicted: [2 2 2 0 0 0 0 0 0 0 0 0]
True:      [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
--------------------------------------------
Index with mismatch: 31
Predicted: [2 2 2 0 0 0 0 0 0 0 0 0 0 0 0]
True:      [2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
--------------------------------------------
Index with mismatch: 41
Predicted: [2 0 0 2 0 0]
True:      [2, 2, 2, 2, 0, 0]
-----------------------------